# Wizualizacja danych pogodowych

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv
import plotly.express as px

## Przygotowanie danych z bazy

In [2]:
load_dotenv()

True

In [3]:
login = os.getenv("LOGIN")
password = os.getenv("PASSWORD")
db_host = os.getenv("DB_HOST")
port = os.getenv("PORT")
database = os.getenv("DB_NAME")

In [4]:
conn_str = (
    f"mssql+pyodbc://{login}:{password}@{db_host},{port}/{database}"
    "?driver=ODBC+Driver+17+for+SQL+Server"
)

In [5]:
engine = create_engine(conn_str)

In [ ]:
query_depesze = text("""
        SELECT Data,  KierunekWiatru, PredkoscWiatru, TemperaturaPowietrza
        FROM Synop.dbo.Depesze as depesze
        WHERE depesze.Data BETWEEN '2024-01-01' and '2024-12-31'
    """)

In [7]:


with engine.connect() as conn:
    df_depesze = pd.read_sql(query_depesze, conn)


In [15]:
df_srednie_dzienne = (
    df_depesze
    .groupby('Data', as_index=False)  # grupujemy po kolumnie Data (dzień)
    .agg({
        'TemperaturaPowietrza': 'mean',
        'PredkoscWiatru': 'mean',
    })
    .rename(columns={
        'TemperaturaPowietrza': 'SredniaTemp',
        'PredkoscWiatru': 'SredniaPredkoscWiatru'
    })
)


In [16]:
df_srednie_dzienne.head()

,Data,SredniaTemp,SredniaPredkoscWiatru
0,2024-01-01,3.033969,2.966808
1,2024-01-02,2.764544,3.519462
2,2024-01-03,4.085300,4.560424
3,2024-01-04,2.640575,4.896286
4,2024-01-05,-1.219833,3.192067


In [17]:
bins = [0, 2, 5, 8, 12, 17, 25, 35, 50]  # Przykładowe zakresy prędkości
labels = [f"{bins[i]}-{bins[i+1]} m/s" for i in range(len(bins)-1)]
df_srednie_dzienne['PrzedzialPredkosci'] = pd.cut(df_srednie_dzienne['SredniaPredkoscWiatru'], bins=bins, labels=labels, right=False)

In [18]:
df_temp_daily = (
    df_depesze
      .groupby('Data', as_index=False)['TemperaturaPowietrza']
      .mean()
      .rename(columns={'TemperaturaPowietrza': 'SredniaTemp'})
)


In [19]:
df_temp_daily.head()

,Data,SredniaTemp
0,2024-01-01,3.033969
1,2024-01-02,2.764544
2,2024-01-03,4.085300
3,2024-01-04,2.640575
4,2024-01-05,-1.219833


In [ ]:

fig = px.scatter_polar(
    df_grouped,
    r='Czestosc',                 # Promień – liczba wystąpień
    theta='KierunekWiatru',       # Kierunek wiatru (stopnie lub kategoria)
    color='PrzedzialPredkosci',   # Kolor odpowiada przedziałowi prędkości
    animation_frame='Data',       # Każda ramka animacji to inny dzień
    hover_data=['SredniaTemp'],   # Dodatkowo w dymku pokazujemy np. średnią temp.
    color_discrete_sequence=px.colors.sequential.Plasma_r,
    start_angle=90,               # 90 st. na górze (północ)
    direction='clockwise',
    title="Dzienne rozkłady wiatru (róża wiatrów z animacją)"
)


fig.update_layout(
    polar=dict(
        radialaxis=dict(showticklabels=True, ticks=''),
        angularaxis=dict(direction='clockwise')
    ),
    legend_title_text="Prędkość wiatru",
    width=800,
    height=700
)

fig.show()


In [ ]:
fig.show()